In [ ]:
%%capture
!pip install openai

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
num_2_gen=550

In [ ]:
import openai
import csv
import pandas as pd
import os

In [ ]:
def load_env():
    with open('.env', 'r') as f:
        for line in f:
            if line.strip():  # 빈 줄이 아닌 경우만 처리
                key, value = line.strip().split('=')
                os.environ[key] = value

# load .env file
load_env()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
# 1 : senstart
# 2 : wmi
# 3 : correctness
def prompt_gen(test_conf, sentence ='one Korean word :\n초콜릿\nfive clues :\n달콤한 맛이 나는 디저트로, 카카오와 우유, 설탕 등을 첨가해 만든 음식입니다.\n우유에 넣어 코코아로 먹기도 합니다.\n더운 날씨에는 녹을 수 있습니다.\n많이 먹으면 치아가 상할 수 있습니다.\n강아지에게 먹이면 안됩니다.'):
  if test_conf==1:
    prompt = "please generate diverse data like this\n\n@example\n{Starting word : Rock}"
    return prompt
  elif test_conf==2:
    prompt=f"please generate diverse data like this by using Korean words and English template\n\n@example\n{sentence}"
    return prompt
  elif test_conf==3:
    prompt="please generate diverse data like this\n\n@example\n### input :\nA pen sings a melodious tune when pressed against the desk\n\n### output :\nincorrect\n### input :\nThe bird sings a melodious tune\n\n### output :\ncorrect"
    return prompt


In [ ]:
from google.colab import output

In [ ]:
MODEL = "gpt-3.5-turbo"

def run_model(prompt):
  new_data = openai.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "assistant", "content": f"{prompt}"},
      ],
      temperature=1,

  )
  return new_data.choices[0].message.content

In [ ]:
# 1 : senstart
# 2 : wmi
# 3 : correctness
dic_dataset={'dataset':[]}
test_conf=2
for n in range(num_2_gen):
  prompt = prompt_gen(test_conf)
  new_data=run_model(prompt)
  dic_dataset['dataset'].append("### instruction :\nPlease generate a word and five clues associated with it in Korean. The generated word and clues should be relevant and provide insightful hints for guessing.\n\n### output :\n{" + new_data + "}")
  output.clear()
  print('current task :'+str(n))
print('total number of dataset'+str(len(dic_dataset)))


current task :304


KeyboardInterrupt: 

In [ ]:
dataset = pd.DataFrame(dic_dataset)

In [ ]:
for idx,row in enumerate(dataset['dataset']):
  if ('one Korean word' not in row) or ('five clues' not in row) or ('}' not in row):
    dataset.drop(idx, axis=0, inplace=True)

In [ ]:
len(dataset.drop_duplicates(subset=None, keep='first', inplace=False, ignore_index=False))


281

In [ ]:
with open("/content/drive/MyDrive/Colab Notebooks/dataset/regen.csv", "a") as f:
  dataset.to_csv(f, index=False, header=False)

In [ ]:
tmp= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dataset/regen.csv')

In [ ]:
len(tmp)

775

In [ ]:
tmp=tmp.drop_duplicates(subset=None, keep='first', inplace=False, ignore_index=False)

In [ ]:
len(tmp)

772

In [ ]:
with open("/content/drive/MyDrive/Colab Notebooks/dataset/regen.csv", "w") as f:
  tmp.to_csv(f, index=False, header=True)